<h1>Deep Reach</h1>

The following implementation is inspired by <h><a href="https://github.com/SimonBirrell/ppo-continuous-control">ppo-continuous-control</a></h> and <h><a href="https://github.com/higgsfield/RL-Adventure-2">RL-Adventure-2</a></h>.

<h2>Import, configure, and load reacher environment</h2>

In [ ]:
# Import libaries
import math
import random
from collections import namedtuple, deque
import gym
import numpy as np

# Setup Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
print(torch.__version__)

# Setup Cuda or CPU
use_cuda = torch.cuda.is_available()
DEVICE   = torch.device("cuda" if use_cuda else "cpu")

# Start Environment
from unityagents import UnityEnvironment
env = UnityEnvironment(file_name="./unity_simulation/Tennis.x86_64")

<h2>Examine the reacher environement</h2>

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)
# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)
# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))

<h2>Hyperparameters</h2>

In [ ]:
HORIZON = 275           # Range 32 to 5000
DISCOUNT_FACTOR = 0.99  # Range 0.8 to 0.9997
GAE_DISCOUNT = 0.99     # Range 0.8 to 0.9997
GAE_LAMBDA = 0.95       # Range 0.9 to 1
EPOCH_RANGE = 12        # Range 3 to 30
MINIBATCH_SIZE = 64     # Range 4 to 4096
PPO_CLIP_RANGE = 0.15   # Range 0.05 to 0.3 [testing --> 0.05; 0.1; 0.15]
LEARNING_RATE = 3e-4    # Range 0.003 to 5e-6

<h2>Utilities</h2>

In [4]:
# Following is inspired/copied from:
# https://github.com/openai/baselines/blob/master/baselines/common/running_mean_std.py    
class RunningMeanStd(object):
    def __init__(self, epsilon=1e-4, shape=()):
        self.mean = np.zeros(shape, 'float64')
        self.var = np.ones(shape, 'float64')
        self.count = epsilon

    def update(self, x):
        batch_mean = np.mean(x, axis=0)
        batch_var = np.var(x, axis=0)
        batch_count = x.shape[0]
        self.update_from_moments(batch_mean, batch_var, batch_count)

    def update_from_moments(self, batch_mean, batch_var, batch_count):
        self.mean, self.var, self.count = update_mean_var_count_from_moments(
            self.mean, self.var, self.count, batch_mean, batch_var, batch_count)    
        
def update_mean_var_count_from_moments(mean, var, count, batch_mean, batch_var, batch_count):
    delta = batch_mean - mean
    tot_count = count + batch_count

    new_mean = mean + delta * batch_count / tot_count
    m_a = var * count
    m_b = batch_var * batch_count
    M2 = m_a + m_b + np.square(delta) * count * batch_count / tot_count
    new_var = M2 / tot_count
    new_count = tot_count

    return new_mean, new_var, new_count

def random_sample(indices, batch_size):
    indices = np.asarray(np.random.permutation(indices))
    batches = indices[:len(indices) // batch_size * batch_size].reshape(-1, batch_size)
    for batch in batches:
        yield batch
    r = len(indices) % batch_size
    if r:
        yield indices[-r:]

class MeanStdNormalizer():
    def __init__(self):
        self.rms = None
        self.clip = 10.0
        self.epsilon = 1e-8

    def __call__(self, x):
        x = np.asarray(x)
        if self.rms is None:
            self.rms = RunningMeanStd(shape=(1, ) + x.shape[1:])
        self.rms.update(x)
        return np.clip((x - self.rms.mean) / np.sqrt(self.rms.var + self.epsilon),
                       -self.clip, self.clip)

<h2>Neural Network (actor/critic)</h2>

In [9]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)
        

class ActorCritic(nn.Module):
    def __init__(self, num_agents, state_size, action_size, hidden_size, seed):
        super(ActorCritic, self).__init__()
        
        self.seed = random.seed(seed)
        
        self.critic = nn.Sequential(
            nn.Linear(state_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(state_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, action_size),
        )
        
        self.log_std = nn.Parameter(torch.ones(1, action_size))
        self.apply(init_weights)
        
    def forward(self, obs, action=None):
        obs = torch.tensor(obs, device=DEVICE, dtype=torch.float32)
        a = self.actor(obs)
        v = self.critic(obs)
        dist = torch.distributions.Normal(a, self.log_std.exp().expand_as(a))
        return (v, dist)

<h2>Rollout manager</h2>

In [5]:
class Rollout_manager():
    
    def __init__(self):
        # Stored data
        self.actions = []
        self.log_prob_actions = []
        self.values = []
        self.rewards = []
        self.episode_not_dones = []
        self.states = []
        
        # Calculated data
        self.returns = [0.0] * HORIZON
        self.advantages = [0.0] * HORIZON
        
    def save_prediction(self, actions, log_prob_actions, values):
        self.actions.append(actions)
        self.log_prob_actions.append(log_prob_actions)
        self.values.append(values)

    def save_consequences(self, rewards, episode_not_dones, states):
        self.rewards.append(rewards)
        self.episode_not_dones.append(episode_not_dones)
        self.states.append(states)
        
    def calculate_returns_and_advantages(self, final_reward):
        self.rewards.append(None)
        self.episode_not_dones.append(None)
        self.calculate_future_returns(final_reward)
        self.estimate_advantages()

    def calculate_future_returns(self, returns):
        for i in reversed(range(HORIZON)):
            returns = self.rewards[i] + DISCOUNT_FACTOR * self.episode_not_dones[i] * returns
            self.returns[i] = returns.detach() 

    def estimate_advantages(self):
        advantages = torch.tensor(np.zeros((num_agents, 1)), device=DEVICE, dtype=torch.float32)
        for i in reversed(range(HORIZON)):
            td = self.rewards[i] + (GAE_DISCOUNT * self.episode_not_dones[i] * self.values[i + 1]) - self.values[i]
            advantages = advantages * GAE_LAMBDA * GAE_DISCOUNT * self.episode_not_dones[i] + td
            self.advantages[i] = advantages.detach()               

    def stack_tensor(self, some_list):
        return torch.cat(some_list[:HORIZON], dim=0)
            
    def get_data(self):
        states = self.stack_tensor(self.states)
        actions = self.stack_tensor(self.actions) 
        log_prob_actions = self.stack_tensor(self.log_prob_actions)
        returns = self.stack_tensor(self.returns)
        advantages = self.stack_tensor(self.advantages)
        advantages = (advantages - advantages.mean()) / advantages.std()        
        return (states, actions, log_prob_actions, returns, advantages)

<h2>PPO algorithm</h2>

In [6]:
class PPO_algorithm():   
    
    def __init__(self, num_agents, state_size, action_size, hidden_size, seed):
        self.network = ActorCritic(num_agents, state_size, action_size, hidden_size, seed).to(DEVICE)
        self.first_states = True
        self.state_normalizer = MeanStdNormalizer()
        
    def evaluate_actions_against_states(self, states, actions):
        value, action_distribution = self.network(states, actions)
        log_prob = action_distribution.log_prob(actions).sum(-1).unsqueeze(-1)
        return (log_prob, value)
    
    def get_prediction(self, states):
        if self.first_states:
            self.states = states
            self.first_states = False
        self.latest_values, action_distribution = self.network(self.states)
        self.latest_actions = action_distribution.sample()
        self.latest_log_prob = action_distribution.log_prob(self.latest_actions).sum(-1).unsqueeze(-1)
        return self.latest_actions
    
    def step(self, states, actions, rewards, next_states, dones):
        rewards = np.asarray(rewards)
        next_states = self.state_normalizer(next_states)
        self.rollout.save_prediction(self.latest_actions, self.latest_log_prob, self.latest_values)
        dones = np.asarray(dones).astype(int)
        rewards = torch.tensor(rewards, device=DEVICE, dtype=torch.float32).unsqueeze(-1)
        episode_not_dones = torch.tensor((1 - dones), device=DEVICE, dtype=torch.float32).unsqueeze(-1)
        states = torch.tensor(self.states, device=DEVICE, dtype=torch.float32)      
        self.rollout.save_consequences(rewards, episode_not_dones, states)
        self.states = next_states
                
    def create_rollout(self):
        self.rollout = Rollout_manager()
            
    def process_rollout(self, states):
        # save final results
        self.get_prediction(states)
        self.rollout.save_prediction(self.latest_actions, self.latest_log_prob, self.latest_values)
        # Calculate resutns and advantages
        self.rollout.calculate_returns_and_advantages(self.latest_values.detach())
        # Optimize
        self.optimize()
        self.first_states = True
   
    def save_weights(self):
        torch.save(self.network.state_dict(), "trained_weights.pth")
        print("Weights are saved to trained_weights.pth")
        
    def ppo_update(self, optimizer, sampled_states, sampled_actions, sampled_log_probs_old, sampled_returns, sampled_advantages):
        log_prob_action, value = self.evaluate_actions_against_states(sampled_states, sampled_actions)
        
        # Policy Loss
        ratio = (log_prob_action - sampled_log_probs_old).exp() 
        obj = ratio * sampled_advantages
        obj_clipped = ratio.clamp(1.0 - PPO_CLIP_RANGE, 1.0 + PPO_CLIP_RANGE) * sampled_advantages
        policy_loss = -torch.min(obj, obj_clipped).mean() 
        
        # Value Loss
        value_loss = 0.5 * (sampled_returns - value).pow(2).mean()
        
        # Optimize network weights
        loss = policy_loss + value_loss
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.network.parameters(), 0.75) 
        optimizer.step()

    def optimize(self):
        states, actions, log_probs_old, returns, advantages = self.rollout.get_data()
        actions = actions.detach()
        log_probs_old = log_probs_old.detach()
        optimizer = torch.optim.Adam(self.network.parameters(), LEARNING_RATE, eps=1e-5)
        for i in range(EPOCH_RANGE):
            number_timesteps = states.size(0)
            timesteps_to_sample = random_sample(np.arange(number_timesteps), MINIBATCH_SIZE) 
            for timestep in timesteps_to_sample:
                t = torch.tensor(timestep, device=DEVICE, dtype=torch.float32).long()
                sampled_states = states[t]
                sampled_actions = actions[t]
                sampled_log_probs_old = log_probs_old[t]
                sampled_returns = returns[t]
                sampled_advantages = advantages[t]
                self.ppo_update(optimizer, sampled_states, sampled_actions, sampled_log_probs_old, sampled_returns, sampled_advantages)

<h1>Train agent</h1>

In [7]:
class train_agent():
    
    def __init__(self, num_workers):
        self.num_workers = num_workers
        self.online_rewards = np.zeros(num_workers)
        self.running_mean_score = 0
        self.mean_scores = []
                
    def train_ppo(self, agent, target_avg_score, max_episodes=300, train_mode_=True):
        if train_mode_:
            print("========================================================================================")
            print("Beginning training...")
            print("GOAL: Mean episode score of {:.2f} over the last 100 episodes in less than {} episodes.".format(target_avg_score, max_episodes))
            print("========================================================================================")
        else:
            print("Testing Agent")
        
        # Reset environment:
        env_info = env.reset(train_mode=train_mode_)[brain_name]
        
        # Initialize variables:
        self.num_episodes = 0
        self.last_100_scores = deque(maxlen=100)
        self.episode_scores = [list() for i in range(num_agents)]
        self.episodes_finished = 0
        self.target_avg_score = target_avg_score
        
        while True:
            # Run a rollout (max HORIZON)
            states = env_info.vector_observations # Get current states
            scores = np.zeros(num_agents) # Initialize scores list
            agent.create_rollout() # Create rollout instance
            
            horizon_ = HORIZON

            for t in range(horizon_):
                # Get actions from policy
                actions = np.clip(agent.get_prediction(states).cpu().detach().numpy(), -1, 1) # Limit max torque between -1 and 1

                # Use the actions in the environemnt and get next state
                env_info = env.step(actions)[brain_name]
                next_states = env_info.vector_observations
                rewards = env_info.rewards
                dones = env_info.local_done

                # Save rewards
                self.online_rewards += rewards
                for i, done in enumerate(dones):
                    # If a worker is done end the episode for this worker
                    if done:
                        # Save the workers reward from the episode
                        self.episode_scores[i].append(self.online_rewards[i])
                        self.episodes_finished +=1
                        # If all workers finished end the entire episode
                        if (self.episodes_finished % num_agents) == 0:
                            max_score = -99.0
                            self.num_episodes += 1
                            # Add up the rewards that each agent received and pick max
                            for j in range(num_agents):
                                single_agent_acore = self.episode_scores[i][-1]
                                max_score = single_agent_acore if single_agent_acore > max_score else max_score
                            self.last_100_scores.append(max_score)
                            self.mean_scores.append(max_score)
                            if train_mode_:
                                print("Episode: %3.0f | Mean max score: %7.4f | Mean 100 episode score: %7.4f" % (self.num_episodes, round(max_score, 6), round(np.mean(self.last_100_scores),6)))
                        
                        env_info = env.reset(train_mode=train_mode_)[brain_name]
                        # Set accumulated reward to zero for next episode
                        self.online_rewards[i] = 0
                        # Calculate running mean
                        self.running_mean_score = np.mean(self.last_100_scores)
                        # Break if the target if reached 
                        if self.running_mean_score > self.target_avg_score and train_mode_:
                            print(" ")
                            print("*** Target reached! ***")
                            print(" ")
                            break

                agent.step(states, actions, rewards, next_states, dones) 
                scores += rewards
                states = next_states
                
            # Optimize the agent using PPO and the data from the last rollout
            agent.process_rollout(states)

            # Check exit conditions
            if self.running_mean_score > target_avg_score and train_mode_:
                print("Mean episode max score of %s over the last 100 episodes" % self.running_mean_score)
                agent.save_weights() 
                break
            if self.num_episodes > max_episodes and train_mode_:
                print("Failed mean episode max score of {:.2f} over the 100 most recent episodes in less than {} episodes.".format(target_avg_score, self.num_episodes))
                agent.save_weights() 
                break
        return self.mean_scores           

<h1> Train PPO agent </h1>

In [ ]:
from numpy import savetxt
import warnings
warnings.filterwarnings('ignore') # ignore Rimtime Warnings

scores        = []
for i in range(5):
    agent         = PPO_algorithm(num_agents, state_size=state_size, action_size=action_size, hidden_size=32, seed=123)
    train_session = train_agent(num_agents)
    scores.append(train_session.train_ppo(agent, 100, 2500))
    
# Save scores to csv file:
episodes = []
scores_con = []
for i in range(len(scores)):
    episodes += range(len(scores[i]))
    scores_con += scores[i]
    
savetxt('./score.csv', np.c_[episodes,scores_con], delimiter=",")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
sns.set()

# Setup seaborn for plotting
sns.set("talk", font_scale=1.5,  rc={"lines.linewidth": 3, 'figure.figsize': (14, 7)})
sns.set_palette("muted")
sns.set_style("whitegrid")

# Get and arrange data
df = pd.read_csv("./score.csv")      # Read CSV file generated while training
df.columns = ["episodes", "scores"]  # Rename columns 

# Plot data
sns_plot = sns.lineplot('episodes', 'scores', data=df)
sns_plot.set(ylabel='Score', xlabel='Episode')
figure = sns_plot.get_figure()
figure.tight_layout()
plt.show()

<h1> Plot scores </h1>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
sns.set()

# Setup seaborn for plotting
sns.set("talk", font_scale=1.5,  rc={"lines.linewidth": 3, 'figure.figsize': (14, 7)})
sns.set_palette("muted")
sns.set_style("whitegrid")
 
# Get and arrange data
df05 = pd.read_csv("./data/score_0_05.csv")      # Read CSV file generated while training
df05.columns = ["episodes", "scores"]  # Rename columns 
df_scores_smooth05 = df05.groupby(["episodes"], as_index=False).mean()        # Prepare for rolling mean
df_scores_smooth05['smooth_score'] = df_scores_smooth05['scores'].rolling(100).mean() # Use rolling mean of 15

df1 = pd.read_csv("./data/score_0_1.csv")      # Read CSV file generated while training
df1.columns = ["episodes", "scores"]  # Rename columns
df_scores_smooth1 = df1.groupby(["episodes"], as_index=False).mean()        # Prepare for rolling mean
df_scores_smooth1['smooth_score'] = df_scores_smooth1['scores'].rolling(100).mean() # Use rolling mean of 15

df15 = pd.read_csv("./data/score_0_15.csv")      # Read CSV file generated while training
df15.columns = ["episodes", "scores"]  # Rename columns 
df_scores_smooth15 = df15.groupby(["episodes"], as_index=False).mean()        # Prepare for rolling mean
df_scores_smooth15['smooth_score'] = df_scores_smooth15['scores'].rolling(100).mean() # Use rolling mean of 15


# Plot data
sns_plot = sns.lineplot('episodes', 'scores', data=df05, linewidth = 1, color="green", alpha = 0.5, label='_nolegend_')
sns_plot = sns.lineplot('episodes', 'scores', data=df1, linewidth = 1, color="orange", alpha = 0.5, label='_nolegend_')
sns_plot = sns.lineplot('episodes', 'scores', data=df15, linewidth = 1, color="blue", alpha = 0.5, label='_nolegend_')

sns_plot = sns.lineplot('episodes', 'smooth_score', data=df_scores_smooth05, linewidth = 3.5, color="green")
sns_plot = sns.lineplot('episodes', 'smooth_score', data=df_scores_smooth1, linewidth = 3.5, color="orange")
sns_plot = sns.lineplot('episodes', 'smooth_score', data=df_scores_smooth15, linewidth = 3.5, color="blue")

sns_plot.set(ylabel='Score', xlabel='Episode')
sns_plot.legend(fontsize = 'small', loc='top left', labels=["PPO_CLIP_RANGE=0.05", "PPO_CLIP_RANGE=0.10", "PPO_CLIP_RANGE=0.15"], framealpha = 0)
figure = sns_plot.get_figure()
figure.tight_layout()
figure.savefig('./score_episode.png', dpi=200) # Save figure to score.png
plt.show()

<h1> Load and test trained agent </h1>

In [10]:
agent         = PPO_algorithm(num_agents, state_size=state_size, action_size=action_size, hidden_size=32, seed=123)
agent.network.load_state_dict(torch.load('./saved_models/trained_weights.pth'))
train_session = train_agent(num_agents)
train_session.train_ppo(agent, 99.0, 300, False)

Testing Agent


/home/mat/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mat/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 